[View in Colaboratory](https://colab.research.google.com/github/Angordil/Analisis-de-registros-de-EEG-mediante-redes-neuronales/blob/master/Data_preparation_All_subjects.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [0]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 29 14:02:48 2018

@author: anaelle
"""
from __future__ import division

from scipy.io import loadmat

subjects_count = 10
subjects = []
for i in range(0, subjects_count):
    filename = "S" + str(i+1) + ".mat"
    subjects.append(loadmat("drive/TFG/data/" + filename))
   
subjects[0]


# Inspect the EEG data for the first trial of the first subject:

trial1 = subjects[0]['X_2D'][0]
print(trial1.shape)
print(trial1)

get_ipython().run_line_magic('matplotlib','inline')

import matplotlib.pyplot as plt

N = 32
plt.plot(trial1[:N])
plt.title("EEG Reading: Electrode 1, Trial 1, Subject 1")

import numpy as np

electrodes = 124
trial_image = np.reshape(trial1, (electrodes, N))
plt.imshow(trial_image, cmap = 'hot', interpolation = 'nearest')

# ### Data Preparation

# Concatenate all trials and output target labels from the first 9 test subjects

trials = np.concatenate([subjects[i]['X_2D'] for i in range(9)])
y_training = np.concatenate([subjects[i]['categoryLabels'][0] for i in range(9)])

# Double-check number of trials equals number of target output labels 

print(len(trials),len(y_training))
print(y_training)

trials[0]

len(trials[0])

# ### Define Training and Validation/Holdout Sets

X_training = trials

holdout_fold = subjects[9]
X_validation = holdout_fold['X_2D']
print(len(X_validation), X_validation)

y_validation = holdout_fold['categoryLabels'][0]
print(len(y_validation),y_validation)
np.save('drive/TFG/validation/y_val', y_validation)

 ## 1-Hot Encode Target Output Labels
import tensorflow
from tensorflow import keras
from tensorflow.python.keras import utils 


num_classes = 6
y_training1hot = utils.to_categorical(y_training - 1 , num_classes)
y_validation1hot = utils.to_categorical(y_validation -1, num_classes)

print y_training1hot.shape
print y_validation1hot.shape

np.save('drive/TFG/training/y_training1hot', y_training1hot)
np.save('drive/TFG/validation/y_validation1hot', y_validation1hot)

In [0]:
# Resize Training and Validation Sets

import numpy as np

X_training = np.reshape(X_training,(-1, electrodes, N,1))
X3_training = np.repeat(X_training,3, axis=3)
np.save('drive/TFG/training/X3_train',X3_training)

X_validation = np.reshape(X_validation, (-1, electrodes, N, 1))
X3_validation = np.repeat(X_validation, 3, axis=3)
np.save('drive/TFG/validation/X3_val', X3_validation)

print(X3_training.shape)